In [4]:
import os
import glob
import numpy as np
import cv2
import tifffile as tiff

def Panoptic_quality(ground_truth_image, predicted_image):
    TP = 0; FP = 0; FN = 0; sum_IOU = 0
    matched_instances = {}

    for i in np.unique(ground_truth_image):
        if i == 0: continue
        temp_image = (ground_truth_image == i)
        matched_image = temp_image * predicted_image
        
        for j in np.unique(matched_image):
            if j == 0: continue
            pred_temp = (predicted_image == j)
            intersection = np.sum(temp_image & pred_temp)
            union = np.sum(temp_image | pred_temp)
            IOU = intersection / union
            if IOU > 0.5:
                matched_instances[i] = (j, IOU)
                    
    pred_indx_list = np.unique(predicted_image)
    pred_indx_list = np.array(pred_indx_list[1:])

    for indx in np.unique(ground_truth_image):
        if indx == 0: continue
        if indx in matched_instances.keys():
            pred_indx_list = np.delete(pred_indx_list, np.argwhere(pred_indx_list == matched_instances[indx][0]))
            TP += 1
            sum_IOU += matched_instances[indx][1]
        else:
            FN += 1
            
    FP = len(np.unique(pred_indx_list))
    denominator = TP + 0.5 * FP + 0.5 * FN
    if denominator == 0: return np.nan
    
    return sum_IOU / denominator
# ---------------------------------------------

pred_dir = 'predicted_masks'
gt_dir = 'ground_truth_masks_combined'

class_dict = {1: 'Epithelial', 2: 'Lymphocyte', 3: 'Macrophage', 4: 'Neutrophil'}
pq_results_per_class = {class_name: [] for class_name in class_dict.values()}

predicted_files = glob.glob(os.path.join(pred_dir, '*.tif'))
print(f"Starte Evaluierung für {len(predicted_files)} Bilder...\n")

for pred_path in predicted_files:
    filename = os.path.basename(pred_path)
    gt_path = os.path.join(gt_dir, filename)
    
    if not os.path.exists(gt_path):
        continue
    
    pred_semantic = tiff.imread(pred_path)
    #print("s", np.unique(pred_semantic))
    gt_semantic = tiff.imread(gt_path)
    #print("gt", np.unique(gt_semantic))

    for class_id, class_name in class_dict.items():
        # Binäre Masken für diese Klasse
        pred_class_mask = (pred_semantic == class_id).astype(np.uint8)
        gt_class_mask = (gt_semantic == class_id).astype(np.uint8)
        
        # Instanzen finden (Zusammenhängende Zellen)
        _, pred_instances = cv2.connectedComponents(pred_class_mask)
        _, gt_instances = cv2.connectedComponents(gt_class_mask)
        
        has_gt = np.max(gt_instances) > 0
        has_pred = np.max(pred_instances) > 0
        
        if has_gt or has_pred:
            pq_value = Panoptic_quality(gt_instances, pred_instances)
            if not np.isnan(pq_value):
                pq_results_per_class[class_name].append(pq_value)

print("--- Panoptic Quality (PQ) Ergebnisse ---")
all_pqs = []
for class_name, pq_list in pq_results_per_class.items():
    if len(pq_list) > 0:
        mean_pq = np.mean(pq_list)
        all_pqs.append(mean_pq)
        print(f"{class_name.ljust(15)}: {mean_pq:.4f} (basierend auf {len(pq_list)} Bildern)")
    else:
        print(f"{class_name.ljust(15)}: Keine Zellen gefunden.")

print("-" * 40)
print(f"Gesamt Mean PQ : {np.mean(all_pqs):.4f}")

Starte Evaluierung für 42 Bilder...

--- Panoptic Quality (PQ) Ergebnisse ---
Epithelial     : 0.0420 (basierend auf 38 Bildern)
Lymphocyte     : 0.1005 (basierend auf 37 Bildern)
Macrophage     : 0.0178 (basierend auf 31 Bildern)
Neutrophil     : 0.1615 (basierend auf 31 Bildern)
----------------------------------------
Gesamt Mean PQ : 0.0805
